In [ ]:
import heapq
from collections import defaultdict, deque
from functools import cache
from typing import Dict, List, Tuple

**62. Unique Paths**

### Problem Statement

Given a grid with `m` rows and `n` columns, you need to find the number of unique paths from the top-left corner to the bottom-right corner. You can only move either down or right at any point in time.

### Combinatorics Approach

The problem can be solved using combinatorics by recognizing that the task is to choose a sequence of moves (right and down) to reach the destination.

1. **Total Moves**:

    - To go from the top-left to the bottom-right corner, you need to make a total of `m-1` down moves and `n-1` right moves.
    - Therefore, the total number of moves required is `(m-1) + (n-1) = m + n - 2`.

2. **Choosing Moves**:

    - Out of these `m + n - 2` moves, you need to choose `m-1` moves to be down (or equivalently, `n-1` moves to be right).
    - The number of ways to choose `m-1` down moves from `m + n - 2` total moves is given by the binomial coefficient `C(m+n-2, m-1)`.

3. **Binomial Coefficient**:
    - The binomial coefficient `C(a, b)` is calculated as `a! / (b! * (a - b)!)`, where `!` denotes factorial.

### Implementation

Here is the Python implementation of the combinatorics solution:

```python
from math import comb

def uniquePaths(m: int, n: int) -> int:
    # Calculate the binomial coefficient C(m+n-2, m-1)
    return comb(m + n - 2, m - 1)
```


**The for loop is doing this:**

$$[ \frac{m \times (m+1) \times \ldots \times (m+n-2)}{1 \times 2 \times \ldots \times (n-1)} ]$$

$$=[ \frac{(m+n-2)!}{(m-1)! \cdot (n-1)!} ]$$


In [2]:
class Solution:
    def uniquePaths(self, m: int, n: int) -> int:  # 100% time, 20% memory
        if m == 1 or n == 1:
            return 1
        if m < n:
            m, n = n, m

        res = j = 1
        for i in range(m, m + n - 1):
            res *= i
            res //= j
            j += 1

        return res

    def uniquePaths2dDp(self, m: int, n: int) -> int:  # 100% time, 9% memory
        """
        The actual DP way is pretty simple I imagine:
        - you have a matrix of size mxn
        - you initialize the (0,0) with 1 and everything else with 0
        - then `dp[i][j] = dp[i-1][j] + dp[i][j-1]` since you can only move in those two directions
        - return `dp[m-1][n-1]`

        Parameters:
        - i
        - j

        Bounds:
        - i [0, m-1]
        - j [0, n-1]

        Direction: low to high for both

        Dependencies:
        For each pair (i,j) you need all pairs (i-1, j) and (i, j-1) to be filled
        So you only need to keep track of either the previous row or the previous col

        I will not use a matrix in this case since I can fix i or j and use an array.
        Just fix the larger one and iterate through the smaller one:
        - O(min(m,n)) space
        - O(m*n) time no matter what since each square needs to be visited

        NOTE: I can remove an iteration if i initalize `dp1 = 1 * [n]`
        """
        # will only store the previous row
        dp1 = [0] * n  # previous row (n columns)
        dp1[0] = 1
        for _ in range(m):
            # copy the leftmost so that I can skip the first column for iteration
            dp0 = dp1[:]  # current row
            for j in range(1, n):
                # dp[i][j] = dp[i-1][j] + dp[i][j-1]
                dp0[j] = dp1[j] + dp0[j - 1]
                # update previous row to the new row
                dp1[j] = dp0[j]

            # moved the update to the loop, under the assumption that overwriting an index is faster than reallocating a new array
            # dp1 = dp0[:]
        return dp0[-1]

    def uniquePaths2dDpSpaceOptimized(
        self, m: int, n: int
    ) -> int:  # 100% time, 9% memory
        """
        I'm looking at my solution right above and I am noticing that I don't need 2 arrays
        I can just overwrite a single array:
        If given a matrix: `dp[i][j] = dp[i-1][j] + dp[i][j-1]`
        I only need to store an array and this becomes: `dpi[j] = dpi[j] + dpi[j-1]`
        This works because until dpi[j] is overwritten to dp[i][j] it is actually dp[i-1][j]
        """

        dpi = [1] * n  # init top row with n cols
        for _ in range(m - 1):
            # already did the top row so no need for m iterations
            for j in range(1, n):
                # leftmost column is always going to be 1, so don't mutate it
                dpi[j] += dpi[j - 1]  # see my docstring, I condensed it via the +=
        return dpi[n - 1]


uniquePaths = Solution()
print(uniquePaths.uniquePaths(99, 30))
print(uniquePaths.uniquePaths2dDp(99, 30))
print(uniquePaths.uniquePaths2dDpSpaceOptimized(99, 30))

36144551783008652956414410375
36144551783008652956414410375
36144551783008652956414410375


**1143. Longest Common Subsequence**

A subsequence of a string is a new string generated from the original string with some characters (can be none) deleted without changing the relative order of the remaining characters.


In [3]:
class Solution:
    def longestCommonSubsequence(
        self, text1: str, text2: str
    ) -> int:  # 10% time, 10% memory
        """
        Given the two strings
        the result for LCS at position i of both strings is as follows:
        if the letters match:
            1 + LCS at position i+1 of both strings
        else:
            0 + max(
                LCS at position i of left string and i+1 of right string,
                LCS at position i+1 of left string and i of right string
            )

        Parameters:
        - ptr1 the position being evaluated for text1
        - ptr2 the position being evaluated for text2

        Bounds:
        - [0, len(text1))
        - [0, len(text2))

        Cache:
        all pairs (ptr1, ptr2)
        O(n * m)
        """
        memo = {}

        def solve(ptr1: int, ptr2: int) -> int:
            if (ptr1, ptr2) in memo:
                return memo[(ptr1, ptr2)]
            elif ptr1 == len(text1) or ptr2 == len(text2):
                # out of bounds:
                # LCS of a string and an empty string is empty string of length zero
                return 0  # len("")
            elif text1[ptr1] == text2[ptr2]:
                answer = 1 + solve(ptr1 + 1, ptr2 + 1)
            else:
                answer = max(
                    solve(ptr1 + 1, ptr2),
                    solve(ptr1, ptr2 + 1),
                )
            memo[(ptr1, ptr2)] = answer
            return answer

        return solve(0, 0)

    def longestCommonSubsequenceDP(
        self, text1: str, text2: str
    ) -> int:  # 84% time, 55% memory
        """
        Bottom up version of my top down memoization
        """
        # initalize an (n+1)x(m+1) grid, the padding is for the empty string and to prevent needing an if statement
        dp = [[0] * (len(text2) + 1) for _ in range(len(text1) + 1)]

        # same logic I explained for top down, but we are building up instead of popping up after going down
        for row in reversed(range(len(text1))):
            for col in reversed(range(len(text2))):
                if text1[row] == text2[col]:
                    dp[row][col] = 1 + dp[row + 1][col + 1]
                else:
                    dp[row][col] = max(dp[row + 1][col], dp[row][col + 1])

        return dp[0][0]

    def longestCommonSubsequenceDPSpaceOptimized(self, text1: str, text2: str) -> int:
        if len(text1) < len(text2):
            text1, text2 = text2, text1

        dp = [0] * (len(text2) + 1)

        for i in range(len(text1) - 1, -1, -1):
            prev = 0
            for j in range(len(text2) - 1, -1, -1):
                # since j is decrementing, in the next iteration this dp[j] becomes previous row dp[j+1]
                temp = dp[j]
                if text1[i] == text2[j]:
                    dp[j] = 1 + prev  # see comment right above
                else:
                    # since dp[i][j] = max(dp[i+1][j], dp[i][j+1])
                    # and i is fixed while j moves
                    # dp[j] is actually dp[i+1][j] until it is visited
                    # so the following expression is valid and equal to the one right above
                    dp[j] = max(dp[j], dp[j + 1])
                prev = temp  # store dp[i+1][j+1] (from next iteration's perspective)

        return dp[0]


longestCommonSubsequence = Solution()
longestCommonSubsequence.longestCommonSubsequenceDP("abcde", "ace")

3

**123. Best Time to Buy and Sell Stock III**

Can only buy and sell 2 times, and can only hold one share at a given time. Have to sell before buying again.

2 possible ways to do it:

-   Recursive DFS with State Machine and Memoization
    -   States: ["Bought", "No State"]
    -   State Transitions: ["Buy", "Skip", "Sell"]
    -   Map:
        -   No State $\stackrel{\text{Buy}}\rightarrow$ Bought
        -   No State $\stackrel{\text{Skip}}\rightarrow$ No State
        -   Bought $\stackrel{\text{Sell}}\rightarrow$ No State
        -   Bought $\stackrel{\text{Skip}}\rightarrow$ Bought
-   Divide and Conquer
    -   Move a threshold `i` from left to right, computing the difference between global max and global min on both sides of the window then storing that in `arr[i]`
    -   Then return the `max(arr)`

I will do both of them for practice


In [4]:
class Solution:
    # 10% time, 7% memory
    def maxProfitStateMachineRecursion(self, prices: List[int]) -> int:
        # Memoization dictionary
        memo: Dict[
            Tuple[int, int, int], int
        ] = {}  # (curDay, transactionsLeft, state) -> maxProfit

        # Helper function to recursively compute the max profit
        def dfs(curDay: int, transactionsLeft: int, state: int) -> int:
            # Base case: no transactions left or we've reached the end of prices
            if transactionsLeft == 0 or curDay == len(prices):
                return 0

            # Check if the result is already memoized
            if (curDay, transactionsLeft, state) in memo:
                return memo[(curDay, transactionsLeft, state)]

            # Option 1: Skip the current day
            skip = dfs(curDay + 1, transactionsLeft, state)

            # Option 2: Transition based on the current state
            if state == 0:  # Currently not holding stock
                # Buy stock, transition to state 1
                buy = dfs(curDay + 1, transactionsLeft, 1) - prices[curDay]
                result = max(skip, buy)
            else:  # Currently holding stock
                # Sell stock, transition to state 0, and use up one transaction
                sell = dfs(curDay + 1, transactionsLeft - 1, 0) + prices[curDay]
                result = max(skip, sell)

            # Store the result in the memo table
            memo[(curDay, transactionsLeft, state)] = result
            return result

        # Start with 2 transactions and no stock held
        return dfs(0, 2, 0)

    # 67% time, 50% memory
    def maxProfitDivideAndConquer(self, prices: List[int]) -> int:
        """
        This divide and conquer algo is kind of greedy, because I am just going to be storing the max possible profit for each window

        I need a suffix max for the right window So that I can just do suffixMax[right window threshold] - prices[right window threshold]

        Overall O(n) time, O(n) space for the suffix array

        It's actually O(2*n) which is equal to O(k*n) because this solution only works for k = 2 transactions allowed
        """

        n = len(prices)
        leftMin, leftMax = (
            prices[0],
            prices[0],
        )  # update these everytime the threshold moves

        rightSuffixMax = [0]
        for i in range(1, n):  # O(n)
            rightSuffixMax.append(max(rightSuffixMax[i - 1], prices[n - i]))
        rightSuffixMax.reverse()  # O(n) I imagine

        # intialization is the threshold right before index 0 such that left window is non existent and right window is the entire array
        left, right = 0, rightSuffixMax[0] - prices[0]
        # if right ends up always being a negative profit, this zero will carry through and we return zero in the end
        res = max(0, right)

        # don't need to iterate n times, because i am treating right window as threshold + 1
        # and when threshold is at n, it is the same as when it is right before 0
        for threshold in range(n - 1):  # O(n)
            # update left pointers
            if prices[threshold] < leftMin:
                leftMin = prices[threshold]
                leftMax = 0  # reset the max pointer since it is now behind the min
            elif prices[threshold] > leftMax:
                leftMax = prices[threshold]
                # no need to update the min pointer since you want it to stay behind the max

            # update the max profit for left and right windows
            left = max(left, leftMax - leftMin)
            right = rightSuffixMax[threshold + 1] - prices[threshold + 1]

            # update result
            res = max(res, left + right)

        return res

    # 51% time, 33% memory
    def maxProfitCopilotInspired(self, prices: List[int]) -> int:
        """
        This is using a kxn matrix
        where the recurrence relation is

        dp[k][i] = MAX of
        - dp[k][i-1]
            - do nothing on day i having done k transactions
        - bestNetWorthUpToDayI + prices[i]
            - profit you can generate if your k^th transaction is to sell on day `i` after making the optimal `k-1` full transactions and kth buy on day `j` where the price is the lowest of all days before `i` and after the day of the last sell from `k-1`th transaction

        `bestNetWorthUpToDayI` needs to be conscious of previous transactions done up to day i (exclusive).
        So, it is the largest profit you can make by day i-1 with k-1 transactions minus the cost of buying on some earlier day

        This is the time reduced solution of my implementation below:
        Goes from O(k*n^2) to O(k*n)
        """
        dp = [[0] * len(prices) for _ in range(3)]

        for k in range(1, 3):
            bestNetWorthUpToDayI = (
                0 - prices[0]
            )  # your bestNetWorthUpToDayI if you buy on day 1 is the negative of the cost of the stock
            for i in range(1, len(prices)):
                dp[k][i] = max(
                    dp[k][
                        i - 1
                    ],  # do nothing on day i having done k transactions by day i-1
                    (
                        bestNetWorthUpToDayI + prices[i]
                    ),  # profit you can generate if your k^th transaction is to sell on day `i` after making the optimal `k-1` full transactions and kth buy on day `j` where the price is the lowest of all days before `i` and after the day of the last sell from `k-1`th transaction
                )

                bestNetWorthUpToDayI = max(
                    bestNetWorthUpToDayI,  # best set of k-1 transactions + kth purchase up to day i (exclusive)
                    (
                        dp[k - 1][i - 1] - prices[i]
                    ),  # best set of k-1 transactions up to day i and then purchase on day i
                )
        return dp[-1][-1]

    # 55% time, 51% memory
    def maxProfitCopilotInspiredSpaceOptimized(self, prices: List[int]) -> int:
        """
        First Optimization:
        - Replace all dp[k-1] with dpk1
        - Replace all dp[k] with dpk

        I don't think I can get rid of the arrays completely
        2nd Opt:
        - Replace dpk[i] with dpki
        - Replace dpk[i-1] with dpki1
        """
        dpk1 = [0] * len(prices)  # k-1 transactions
        dpki1 = dpki = 0  # k transactions on (i-1)th and (i)th day

        for _ in range(1, 3):  # removing the `k` iteration variable since it isn't used
            bestNetWorthUpToDayI = (
                0 - prices[0]
            )  # your bestNetWorthUpToDayI if you buy on day 1 is the negative of the cost of the stock
            for i in range(1, len(prices)):
                dpki = max(
                    dpki1,  # do nothing on day i having done k transactions by day i-1
                    (
                        bestNetWorthUpToDayI + prices[i]
                    ),  # profit you can generate if your k^th transaction is to sell on day `i` after making the optimal `k-1` full transactions and kth buy on day `j` where the price is the lowest of all days before `i` and after the day of the last sell from `k-1`th transaction
                )

                bestNetWorthUpToDayI = max(
                    bestNetWorthUpToDayI,  # best set of k-1 transactions + kth purchase up to day i (exclusive)
                    (
                        dpk1[i - 1] - prices[i]
                    ),  # best set of k-1 transactions up to day i and then purchase on day i
                )

                dpk1[i - 1] = dpki1  # since we won't need it again
                dpki1 = dpki  # dp[k][i] becomes dp[k][i-1] in next iteration

        return dpki

    # Time Limit Exceeded
    def maxProfit(self, prices: List[int]) -> int:
        """
        O(2*n^2) time limit exceeded

        Reccurrence Relation:
        ```
        dp[k, i] = max(dp[k, i-1], prices[i] - prices[j] + dp[k-1, j-1])
        ```
        Where
        - k is the number of transactions {0,1,2}
        - i is the current day (index of prices)
        - j is all earlier days [0, i)

        The issue is that the search space is too large and time limit gets exceeded.

        It is basically saying, the max profit on day `i` if you have `k` transactions is:
        - the max profit of day `i-1` with `k` transactions (this is just not doing anything on day `i`)
        - or the profit you can get from selling on day `i` + the max profit from the day before you would buy in for the newest transaction

        2D-DP (reduced to 2D because j is a dependent of i and is solved in earlier iterations)
        Build solution from k 0 -> 2 and i 0 -> len(prices)
        """

        dp = [[0] * len(prices) for _ in range(3)]

        res = 0
        for numTransactions in range(1, 3):  # can skip 0 transactions across all days
            for i in range(1, len(prices)):
                dp[numTransactions][i] = res  # do nothing on the day

                if prices[i] <= prices[i - 1]:
                    # no profit to be had at an "inflection" point, not really inflection but yeah
                    continue

                for j in range(i):
                    dp[numTransactions][i] = max(
                        (
                            dp[numTransactions][i]
                        ),  # maintain the optimal action on the day
                        (
                            prices[i]
                            - prices[j]
                            + (dp[numTransactions - 1][j - 1] * int(j > 0))
                        ),  # sell on the day after buying on day j
                    )

                res = max(res, *dp[numTransactions])
        return res


maxProfit = Solution()

print(maxProfit.maxProfitStateMachineRecursion([3, 3, 5, 0, 0, 3, 1, 4]))
print(maxProfit.maxProfitCopilotInspiredSpaceOptimized([1, 2, 3, 4, 5]))
print(maxProfit.maxProfitCopilotInspiredSpaceOptimized([7, 6, 4, 3, 1]))
print(maxProfit.maxProfitCopilotInspiredSpaceOptimized([2, 1, 4, 5, 2, 9, 7]))
print(maxProfit.maxProfitCopilotInspiredSpaceOptimized([3, 2, 6, 5, 0, 3]))
print(maxProfit.maxProfitCopilotInspiredSpaceOptimized([0, 8, 5, 7, 4, 7]))

6
4
0
11
7
11


**308. Best Time to Buy and Sell Stock with Cooldown**


In [5]:
class Solution:
    """
    I think the state machine solution from Best Time to Buy and Sell Stock III is the easiest solution for this.
    I'll do that first, but I also want to try the dp way but there's no limit on number of transactions, iterations stop when the result doesn't change I guess.
    """

    # 44% time, 15% memory
    def maxProfitStateMachine(self, prices: List[int]) -> int:
        """
        {1: BOUGHT, 0: NO-STATE}

        O(2n) time and space since the memo key is (i, state) where i : [0, n), state: {0,1}
        """
        memo = {}  # {(curDay, state): totalProfit}

        def dfsMaxProfitPossibleWith(curDay, state) -> int:
            if curDay >= len(prices):
                return 0
            elif (curDay, state) in memo:
                return memo[(curDay, state)]

            # SKIP current day
            skip = dfsMaxProfitPossibleWith(curDay + 1, state)

            if state:
                # In BOUGHT so we can SELL
                # skip one day since there is a cooldown
                stateExclusive = (
                    dfsMaxProfitPossibleWith(curDay + 2, state=0)
                    + prices[curDay]  # sell = make profit
                )
            else:
                # In NO-STATE so we can BUY
                stateExclusive = (
                    dfsMaxProfitPossibleWith(curDay + 1, state=1)
                    - prices[curDay]  # buy = lose profit
                )

            memo[(curDay, state)] = max(stateExclusive, skip)
            return memo[(curDay, state)]

        return dfsMaxProfitPossibleWith(0, 0)

    def maxProfitBottomUpForwardPass(self, prices: List[int]) -> int:
        """
        The maxProfit at the end of day i with k transactions done is the max of
        - the maxProfit at the end of day i-1 with k transactions done
            - this means you are not buying today
        - the maxProfit at the end of day i-2 with k-1 transactions done
            - this means you are buying today
            - Need to keep track of the `highestNetWorthUpToLastBuy` variable though which keeps track of the minimum cost stock in order to determine the optimal buy times

        First implementation: 6% time, 30% memory

        Second implementation: 6% time, 50% memory
        - get rid of the if statement checking if curDay > 1 and just make the first iteration outside the for loop
        """
        if len(prices) == 1:
            # need this check because I am computing day 2 outside of a for loop
            return 0
        dp_k_prev = [0] * len(prices)
        dp_k = [0] * len(prices)

        res = 0
        while True:
            # negative net worth by buying stock on first day in all cases

            dp_k[1] = max(
                (dp_k[0]),  # not buying today
                (-prices[0] + prices[1]),  # selling today
            )
            highestNetWorthUpToLastBuy = -min(prices[:2])

            for curDay in range(2, len(prices)):
                dp_k[curDay] = max(
                    (dp_k[curDay - 1]),  # not buying today
                    (highestNetWorthUpToLastBuy + prices[curDay]),  # selling today
                )

                highestNetWorthUpToLastBuy = max(
                    highestNetWorthUpToLastBuy,  # not buying today
                    (dp_k_prev[curDay - 2] - prices[curDay]),  # buying today
                )

            maxOnKTransactions = max(dp_k)
            if maxOnKTransactions == res:
                # if no change happens after an extra transaction is attempted, then the optimal solution is found
                return res
            res = max(maxOnKTransactions, res)
            dp_k_prev = dp_k[:]

    # 76% time, 21% memory
    def maxProfitBottomUpNeetCodeInspired(self, prices: List[int]) -> int:
        n = len(prices)
        # 2n space: columns are whether or not you bought or sold on that day, rows are the day
        dp = [[0] * 2 for _ in range(n)]

        # need to treat the last day alone because of the iteration
        # if you do not buy on the last day, max profit is selling:
        dp[n - 1][False] = prices[n - 1]
        # if you do buy on last day, max profit is skipping (lol so you never buy on the last day)
        dp[n - 1][True] = 0

        # go through all days
        for curDay in reversed(range(n - 1)):
            # BUY on current day means the sell branch for the next day is most profitable
            buy = dp[curDay + 1][False] - prices[curDay]
            # Cooldown if you aren't buying means you sold or are skipping, in which case you check the buy branch to start your next transaction
            cooldownInsteadOfBuy = dp[curDay + 1][True]
            # Now update the max profit you can get on current day if you aren't selling:
            dp[curDay][True] = max(buy, cooldownInsteadOfBuy)

            # SELL on current day means you need to check the buy branch two days in the future to start your next transaction
            sell = (
                dp[curDay + 2][True] + prices[curDay]
                if curDay + 2 < n
                else prices[curDay]
            )
            # Cooldown if you aren't selling means you are selling in the future so check a future branch
            cooldownInsteadOfsell = dp[curDay + 1][False]
            dp[curDay][False] = max(sell, cooldownInsteadOfsell)

        # then return the max profit from the buying as early as possible branch:
        return dp[0][1]

    # 80% time, 25% memory
    def maxProfitNeetCode(self, prices: List[int]) -> int:
        n = len(prices)
        dp = [[0] * 2 for _ in range(n + 1)]

        for i in range(n - 1, -1, -1):
            for buying in [True, False]:
                if buying:
                    buy = dp[i + 1][False] - prices[i] if i + 1 < n else -prices[i]
                    cooldown = dp[i + 1][True] if i + 1 < n else 0
                    dp[i][1] = max(buy, cooldown)
                else:
                    sell = dp[i + 2][True] + prices[i] if i + 2 < n else prices[i]
                    cooldown = dp[i + 1][False] if i + 1 < n else 0
                    dp[i][0] = max(sell, cooldown)

        return dp[0][1]


maxProfitCooldown = Solution()
print(maxProfitCooldown.maxProfitBottomUpNeetCodeInspired([1, 2, 3, 3, 2, 0, 5, 3]))
print(maxProfitCooldown.maxProfitBottomUpNeetCodeInspired([1]))

7
0


**511. Coin Change II**


In [6]:
class Solution:  # 97% time, 56% memory
    def change(self, amount: int, coins: List[int]) -> int:
        """
        Very similar to ByteDance OA question but that one was much harder lol

        Parameters:
        - a: the current amount of money being 'made'
        - c: the current coin being evaluated

        Bounds:
        - a in [0, amount]
        - c in coins

        The point will be to get how many ways you can sum coins for all values a in [0, amount] as the DP problem
        Given an (a, c) matrix (row is money sum, column is coin being considered (where all earlier columns have already been considered))
        Then the most way to get a sum `a` when at coin index `c` is the following sum:
        NOTE: I am using c as an index in this block for readability
        ```
        dp[a][c] = sum(
            dp[a][c-1], # skip current coin
            dp[a-coins[c]][c], # use current coin
        )
        ```

        Note that since the two terms in the sum are independent, you don't need a matrix
        - One is dependent on the value `a - coin`, the other is dependent on the index of coin `c`

        ```
        for coin in coins:
            for a in range(coin, amount+1):
                dp[a] += dp[a-coin]
        ```
        Since you can use a coin infinite times, you iterate starting from coin in a forward pass so that you can multi-count the same coin
        You have to start from a == coin because negative indices cause problems
        """

        dp = [0] * (amount + 1)
        dp[0] = 1  # there is always one way to make a sum of zero: empty set {}
        for coin in coins:
            for a in range(coin, amount + 1):
                dp[a] += dp[a - coin]

        return dp[amount]


change = Solution()
print(change.change(5, [1, 2, 5]))
print(change.change(3, [2]))
print(change.change(10, [10]))

4
0
1


**494. Target Sum**

### Combinatorics Approach

Given target and an array nums, the problem is asking how many ways you can split nums into a subset P and a subset N such that
$$\sum P + \sum N = \text{target}$$
NOTE: N consists of negative numbers so its sum is negative.

In addition, we know that $$\sum P - \sum N = \sum \text{nums}$$

Adding them you get $$2\cdot \sum P - \sum N + \sum N = \text{target} + \sum \text{nums}$$
Reducing to $$\sum P = \frac{\text{target} + \sum \text{nums}}{2}$$

From here we can a constraint on the sum of target and nums. Since the sum of integers P is another integer, the numerator must be an even number
$$\therefore (target + \sum \text{nums}) \% 2 = 0$$

In addition $\sum \text{nums} >= |\text{target}|$ is necessary since $0 < \sum P <= \sum \text{nums}$

This also means we can reduce it to the following equation `targetSumP = (target + sum(nums)) // 2`. Then it's just a 1d dp subset sum problem.


In [7]:
class Solution:
    # 63% time, 59% memory
    def findTargetSumWays(self, nums: List[int], target: int) -> int:
        """
        Parameters:
        - t: current path's sum
        - i: current index of nums

        Bounds:
        - t in [-sum(nums), sum(nums)]
        - i in [0, len(nums))

        Dependencies:
        - t(i) in `[-sum(nums[:i+1]), sum(nums[:i+1])]` # we include nums[i]
        - i is dependent on i-1 (the state created by the previous number)

        Given a (T,N) matrix `dp` where the rows are path sums `t` and the columns are the indices `i` of `nums`

        Recurrence Relation:
        - The simplest I can see is `dp[t][i] = dp[t - nums[i]][i-1] + dp[t + nums[i]][i-1]`
        - The number of ways to get to the current sum `t` @ idx `i` is the number of ways to get to the sum `t - nums[i]` + `t + nums[i]` @ idx `i-1`
        - The caveat is that the path needs to use every number up to index `i`

        If the whole dp array is initialized as zeroes it should work.
        - But, I think I also have to do `dp[nums[i]][i] += 1` and `dp[-nums[i]][i] += 1` since that is true by definition
        - For clarity, given the value `nums[i]`, you can reach a path sum of `-nums[i]` and `nums[i]` in one more way.

        NOTE: I can immediately tell that I don't need a matrix and that two arrays are enough because i is dependent on i-1 (fixed dependency)
        But I will use a matrix first to see the time and memory gains on leetcode
        """

        sumNums = sum(nums)
        if sumNums < target:
            # early exit base case if it is impossible
            return 0

        nPrev = len(nums)
        nums = [x for x in nums if x != 0]
        numZeros = nPrev - len(nums)

        if not nums:
            # if target is zero and the array is a bunch of zeros
            return 2**numZeros

        dp = [[0] * len(nums) for _ in range(sumNums * 2 + 1)]
        # index 0 through sumNums are for zero and sums > 0
        # index sumNums + 1 through sumNums * 2 are for sums < 0 (these will be accessed with negative indexing)
        # init the first non Zero number option
        dp[-nums[0]][0] += 1
        dp[nums[0]][0] += 1
        sumReached = nums[0]

        for i in range(1, len(nums)):
            num = nums[i]

            for t in range(-sumReached - num, sumReached + num + 1):
                dp[t][i] += (
                    dp[t - num][i - 1]  # if you do '+' nums[i] to reach t
                    + dp[t + num][i - 1]  # if you do '-' nums[i] to reach t
                )

            sumReached += num

        return dp[target][len(nums) - 1] * 2**numZeros

    # 82% time, 64% memory
    def findTargetSumWaysSpaceOptimized(self, nums: List[int], target: int) -> int:
        sumNums = sum(nums)
        if sumNums < target:
            # early exit base case if it is impossible
            return 0

        dp = [0] * (sumNums * 2 + 1)
        dp[0] += 1
        sumReached = 0

        for i in range(0, len(nums)):
            num = nums[i]
            dpCur = [0] * (sumNums * 2 + 1)
            for t in range(-sumReached - num, sumReached + num + 1):
                dpCur[t] += (
                    dp[t - num]  # if you do '+' nums[i] to reach t
                    + dp[t + num]  # if you do '-' nums[i] to reach t
                )

            sumReached += num
            dp = dpCur

        return dp[target]

    def findTargetSumWaysNeetCodeBottomUp(self, nums: List[int], target: int) -> int:
        n = len(nums)
        dpPrev = defaultdict(int)
        dpPrev[0] = 1

        for i in range(n):
            dpCur = defaultdict(int)
            for total, count in dpPrev.items():
                dpCur[total + nums[i]] += count
                dpCur[total - nums[i]] += count
            dpPrev = dpCur

        return dpPrev[target]

    def findTargetSumWaysNeetCodeTopDown(self, nums: List[int], target: int) -> int:
        dp = {}  # (index, total) -> # of ways

        def backtrack(i, total):
            if i == len(nums):
                return 1 if total == target else 0
            if (i, total) in dp:
                return dp[(i, total)]

            dp[(i, total)] = backtrack(i + 1, total + nums[i]) + backtrack(
                i + 1, total - nums[i]
            )
            return dp[(i, total)]

        return backtrack(0, 0)

    # 99.72% time, 62% memory
    def findTargetSumWaysCombinatorics(self, nums: List[int], target: int) -> int:
        sumNums = sum(nums)
        if sumNums < abs(target):
            return 0
        numerator = sumNums + target
        if numerator % 2:
            # sum of positive subset has to be an integer, so the numerator needs to be an even number
            return 0

        targetSumP = numerator // 2
        dp = [1] + [0] * targetSumP  # init 1 way for empty set sum = 0
        for n in nums:
            # Coin change type beat
            for i in reversed(range(len(dp) - n)):
                if dp[i]:
                    dp[i + n] += dp[i]

        return dp[targetSumP]


findTargetSumWays = Solution()
print(findTargetSumWays.findTargetSumWaysCombinatorics([1, 1, 1, 1, 1], 3))
print(findTargetSumWays.findTargetSumWaysCombinatorics([1], 2))
print(findTargetSumWays.findTargetSumWaysCombinatorics([1, 0], 1))
print(findTargetSumWays.findTargetSumWaysCombinatorics([0, 0, 0, 0, 0, 0, 0, 0, 1], 1))
print(
    findTargetSumWays.findTargetSumWaysCombinatorics([7, 9, 3, 8, 0, 2, 4, 8, 3, 9], 0)
)
print(findTargetSumWays.findTargetSumWaysCombinatorics([0, 0, 0, 0, 0, 0], 0))
print(findTargetSumWays.findTargetSumWaysCombinatorics([1, 999], 998))

5
0
2
256
0
64
1


**97. Interleaving String**


In [8]:
class Solution:
    # 13% time. 18% memory
    def isInterleave(self, s1: str, s2: str, s3: str) -> bool:
        """
        Bottom Up solution:
        At each step (i1,i2) you evaluate whether taking character i1 of s1 or i2 of s2 would lead to successfully creating s3
        This is done by checking (i1+1, i2) if you take i1 of s1, or (i1, i2+1) if you take i2 of s2
        - These subproblems will have been solved since we are going bottom up

        Parameters
        - i1 in [0, len(s1)]
        - i2 in [0, len(s2)]

        NOTE: You also need to consider the position (i1, i2) itself, not just if its descendants can work
        """
        if len(s1) + len(s2) != len(s3):
            return False

        dp = [[False] * (len(s2) + 1) for _ in range(len(s1) + 1)]
        dp[len(s1)][len(s2)] = True
        # i1 is the row, i2 is the column
        for i2 in reversed(range(len(s2) + 1)):
            for i1 in reversed(range(len(s1) + 1)):
                if i1 < len(s1) and s1[i1] == s3[i1 + i2] and dp[i1 + 1][i2]:
                    dp[i1][i2] = True
                if (
                    not dp[i1][i2]
                    and i2 < len(s2)
                    and s2[i2] == s3[i1 + i2]
                    and dp[i1][i2 + 1]
                ):
                    # not dp[i1][i2], I put this check just to avoid checking the other branch if the position is already valid
                    dp[i1][i2] = True

        return dp[0][0]


isInterleave = Solution()
print(isInterleave.isInterleave("aabcc", "dbbca", "aadbbcbcac"))
print(isInterleave.isInterleave("aabcc", "dbbca", "aadbbbaccc"))
print(isInterleave.isInterleave("a", "", "c"))
print(isInterleave.isInterleave("", "", ""))

True
False
False
True


**2940. Find Building Where Alice and Bob Can Meet**

The movement from index i to j does not need to meet the constrainst for all indices between i and j. It is not a contiguous movement, it is a hop from one height to the other.


In [9]:
class Solution:  # 90% time, 27% memory
    def leftmostBuildingQueries(
        self, heights: List[int], queries: List[List[int]]
    ) -> List[int]:
        """
        Add the queries to heap in format (maxHeightOfQueryIndices, indexOfQueryInQueriesArray)
        But, if the query can answer itself do not add it to the heap.
        - Ex) [0,0] = 0
        - Ex) [1,5] where `heights[5]` > `heights[1]` = 5

        Trivial problem once these insights are obtained point.
        The heap allows you to solve the problem in one pass through `heights`
        """
        mapQueryRight = defaultdict(list)
        res = [-1] * len(queries)  # failure case is -1 for a query index
        for i, query in enumerate(queries):
            l, r = sorted(query)
            if l == r or heights[r] > heights[l]:
                res[i] = r
                continue
            else:
                mapQueryRight[r].append(
                    (heights[l], i)
                )  # we know that heights[l] >= heights[r]

        minHeapQueries = []
        for i, h in enumerate(heights):
            for query in mapQueryRight[i]:
                heapq.heappush(minHeapQueries, query)

            while minHeapQueries and h > minHeapQueries[0][0]:
                res[heapq.heappop(minHeapQueries)[1]] = i

        return res


leftmostBuildingQueries = Solution()
print(
    leftmostBuildingQueries.leftmostBuildingQueries(
        [6, 4, 8, 5, 2, 7], [[0, 1], [0, 3], [2, 4], [3, 4], [2, 2]]
    )
)
print(
    leftmostBuildingQueries.leftmostBuildingQueries(
        [5, 3, 8, 2, 6, 1, 4, 6], [[0, 7], [3, 5], [5, 2], [3, 0], [1, 6]]
    )
)

[2, 5, -1, 5, 2]
[7, 6, -1, 4, 6]


**329. Longest Increasing Path in a Matrix**


In [10]:
class Solution:  # 93% time, 59% memory
    def longestIncreasingPath(self, matrix: List[List[int]]) -> int:
        """
        DFS and store the answer in a matrix
        """
        ROWS, COLS = len(matrix), len(matrix[0])
        dp = [[0 for _ in range(COLS)] for _ in range(ROWS)]
        dirs = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        def dfs(i, j) -> int:
            if dp[i][j]:
                # already DFSed
                return dp[i][j]

            cur = matrix[i][j]
            tempResult = 0

            for di, dj in dirs:
                ni, nj = i + di, j + dj
                if 0 <= ni < ROWS and 0 <= nj < COLS and matrix[ni][nj] > cur:
                    tempResult = max(tempResult, dfs(ni, nj))

            # add current position to the longest inc path from this position's children
            dp[i][j] = tempResult + 1
            return tempResult + 1

        for i in range(ROWS):
            for j in range(COLS):
                dfs(i, j)

        return max(position for row in dp for position in row)

    def longestIncreasingPathTopologicalSort(self, matrix: List[List[int]]) -> int:
        """
        Kahn's algo for top sort using indegree
        """
        ROWS, COLS = len(matrix), len(matrix[0])
        directions = [[-1, 0], [1, 0], [0, -1], [0, 1]]
        indegree = [[0] * COLS for _ in range(ROWS)]

        for r in range(ROWS):
            for c in range(COLS):
                for d in directions:
                    nr, nc = d[0] + r, d[1] + c
                    if (
                        0 <= nr < ROWS
                        and 0 <= nc < COLS
                        and matrix[nr][nc] < matrix[r][c]
                    ):
                        indegree[r][c] += 1

        q = deque()
        for r in range(ROWS):
            for c in range(COLS):
                if indegree[r][c] == 0:
                    q.append([r, c])

        LIS = 0
        while q:
            for _ in range(len(q)):
                r, c = q.popleft()
                for d in directions:
                    nr, nc = r + d[0], c + d[1]
                    if (
                        0 <= nr < ROWS
                        and 0 <= nc < COLS
                        and matrix[nr][nc] > matrix[r][c]
                    ):
                        indegree[nr][nc] -= 1
                        if indegree[nr][nc] == 0:
                            q.append([nr, nc])
            LIS += 1
        return LIS


longestIncreasingPath = Solution()
print(
    longestIncreasingPath.longestIncreasingPath(
        [
            [9, 9, 4],
            [6, 6, 8],
            [2, 1, 1],
        ]
    )
)
print(
    longestIncreasingPath.longestIncreasingPath(
        [
            [3, 4, 5],
            [3, 2, 6],
            [2, 2, 1],
        ]
    )
)
print(longestIncreasingPath.longestIncreasingPath([[1]]))
print(
    longestIncreasingPath.longestIncreasingPath(
        [
            [7, 8, 9],
            [9, 7, 6],
            [7, 2, 3],
        ]
    )
)

4
4
1
6


**115. Distinct Subsequences**


In [11]:
class Solution:
    # 79% time, 44% memory without removing uncommon letters from s.
    #   Just wanted to test it out lol
    # 78% time, 40% memory with the break in the inner loop
    #   I noticed that the dp matrix is just an upper triangular matrix so no need to loop unnecessarily
    # 39% time, 39% memory without the conditional break in the inner for loop
    def numDistinct(self, s: str, t: str) -> int:
        """
        So I am thinking something very similar to word break or longest common subsequence
        It is essentially longest common subsequence but you are forced to complete a word

        dp matrix of shape len(s) x len(t), initially filled with 0 which I am omitting.

        ex) s = "bagag", t = "bag"
        ```
          b a g
        b[     ]
        a[     ]
        g[     ]
        a[     ]
        g[     ]
        ```

        Start from bottom right and go to top left.
        The positions will store how many successful subsequences pass through the position

        So in the first few iterations:
        NOTE: when you encounter the last letter of t as you traverse up s, you start a new subsequence
        ```
          b a g           b a g           b a g
        b[     ]        b[     ]        b[     ]
        a[     ]        a[     ]        a[     ]
        g[     ]        g[     ]        g[0 0 1]
        a[     ]        a[0 1 0]        a[0 1 0]
        g[0 0 1]        g[0 0 1]        g[0 0 1]
        ```

        And now I am making my first modification:
        - When encountering the first 'a' in `s` which I encounter 2nd after the last 'a' in `s`,
        - I need to have access to the fact that there are two subsequences active for the 'g' in `t`
        - So maybe I only store a single array or I add earlier terms

        Will do:
        - position value + 1 when its the last letter of `t` ('g')
        - position value + the position value of the letter following the current one in `t`

        ```
          b a g           b a g           b a g           b a g           b a g
        b[     ]        b[     ]        b[     ]        b[     ]        b[3 3 2]
        a[     ]        a[     ]        a[     ]        a[0 3 2]        a[0 3 2]
        g[     ]        g[     ]        g[0 1 2]        g[0 1 2]        g[0 1 2]
        a[     ]        a[0 1 1]        a[0 1 1]        a[0 1 1]        a[0 1 1]
        g[0 0 1]        g[0 0 1]        g[0 0 1]        g[0 0 1]        g[0 0 1]
        ```

        Recurrence Relation:
        - if the letter matches:
            - `dp[i][j] = dp[i+1][j] + dp[i+1][j+1]`
            - Unless j+1 is out of bounds then `+ 1`
        - else
            - `dp[i][j] = dp[i+1][j]` (just skip)
        - Test ot see if only works if there are no excess unused letters (uncommon letters).

        Using the recurrence relation with excess letters:
        ```
          b a g
        b[3 3 2]
        a[0 3 2]
        d[0 1 2]
        g[0 1 2]
        a[0 1 1]
        g[0 0 1]
        d[0 0 0]
        ```
        Seems to work!

        """
        # it is overall faster without this, probably because the test cases don't have a lot of uncommon letters
        # lettersOfT = set(t)
        # s = "".join(c for c in s if c in lettersOfT)

        if not s or not t:
            return 0

        ROWS, COLS = len(s), len(t)

        # I am adding 1 to the j out of bounds in order to start new subsequences
        # I am also padding the i out of bounds with zeros to avoid an if statement
        dp = [[0] * COLS + [1] for _ in range(ROWS + 1)]

        for idxS in reversed(range(ROWS)):
            for idxT in reversed(range(COLS)):
                # the int(boolean) is the condensed recurrence relation i describe in docstring
                dp[idxS][idxT] = dp[idxS + 1][idxT] + (
                    dp[idxS + 1][idxT + 1] * int(s[idxS] == t[idxT])
                )

                # if there is no path through here than we simply haven't gotten far into the iteration enough
                # so stop checking
                if dp[idxS][idxT] == 0:
                    break

        return dp[0][0]

    # 88% time, 87% memory
    def numDistinctSpaceOptimized(self, s: str, t: str) -> int:
        """
        dp[idxS] is only reliant on dp[idxS + 1] for all idxT columns
        So we only need one array.
        """
        if not s or not t:
            return 0

        ROWS, COLS = len(s), len(t)

        # I am adding 1 to the j out of bounds in order to start new subsequences
        dp = [0] * COLS + [1]

        for idxS in reversed(range(ROWS)):
            prev = dp[COLS]

            for idxT in reversed(range(COLS)):
                next = dp[idxT]
                dp[idxT] += prev * int(s[idxS] == t[idxT])
                prev = next

                # if there is no path through here than we simply haven't gotten far into the iteration enough
                # so stop checking
                if dp[idxT] == 0:
                    break

        return dp[0]


numDistinct = Solution()
numDistinct.numDistinctSpaceOptimized("bagag", "bag")

3

**72. Edit Distance**

Levenshtein Distance


In [12]:
class Solution:
    # 66% time, 15% memory
    def minDistance(self, word1: str, word2: str) -> int:
        """
        At each position (i,j) you have two conditions:
        - word1[i] == word2[j]
            - zero ops needed, can increment both pointers
        - word1[i] != word2[j]
            - 1 op needed
            - insert into word1: maintain i, increment j
            - delete from word1: increment i, maintain j
            - replace word1[i], increment both
        """
        n, m = len(word1), len(word2)
        # I am padding the bottom row with the base case of matching an empty string to whatever position we are at in word2
        # I am padding the right column with the base case of mathing an empty string to whatever position we are at in word1
        dp = [[0] * m + [i + 1] for i in reversed(range(n))]
        dp.append([j for j in reversed(range(m + 1))])

        # I will do a post order traversal in the sense that when I am at (i,j) I am evaluating how to get to (i,j) not what to do at (i,j)
        for i in reversed(range(n)):
            letter1 = word1[i]
            for j in reversed(range(m)):
                if letter1 == word2[j]:
                    # can go straight from (i+1, j+1) to (i,j) with no op
                    dp[i][j] = dp[i + 1][j + 1]
                else:
                    insert = dp[i][j + 1]
                    delete = dp[i + 1][j]
                    replace = dp[i + 1][j + 1]

                    dp[i][j] = min(insert, delete, replace) + 1

        return dp[0][0]

    # 79.5% time, 93% memory
    def minDistanceSpaceOptimized(self, word1: str, word2: str) -> int:
        n, m = len(word1), len(word2)
        # base case of empty string
        if not min(n, m):
            return max(n, m)

        # I am padding the bottom row with the base case of matching an empty string to whatever position we are at in word2
        dpPrev = list(reversed(range(m + 1)))

        for i in reversed(range(n)):
            # I am padding the right column with the base case of mathing an empty string to whatever position we are at in word1
            # goes from n - (n-1) = 1 to n - 0 = 0 as you move up the rows (same as in my non-optimized solution)
            dp = [0] * m + [n - i]
            letter1 = word1[i]

            for j in reversed(range(m)):
                if letter1 == word2[j]:
                    # can go straight from (i+1, j+1) to (i,j) with no op
                    dp[j] = dpPrev[j + 1]
                else:
                    insert = dp[j + 1]
                    delete = dpPrev[j]
                    replace = dpPrev[j + 1]

                    dp[j] = min(insert, delete, replace) + 1

            dpPrev = dp

        return dp[0]

    # 80.6% time, 92% memory
    def minDistanceOptimalNeetCode(self, word1: str, word2: str) -> int:
        m, n = len(word1), len(word2)
        if m < n:
            m, n = n, m
            word1, word2 = word2, word1

        dp = [n - i for i in range(n + 1)]

        for i in range(m - 1, -1, -1):
            nextDp = dp[n]
            dp[n] = m - i
            for j in range(n - 1, -1, -1):
                temp = dp[j]
                if word1[i] == word2[j]:
                    dp[j] = nextDp
                else:
                    dp[j] = 1 + min(dp[j], dp[j + 1], nextDp)
                nextDp = temp
        return dp[0]


minDistance = Solution()
print(minDistance.minDistanceSpaceOptimized("lemon", "mono"))
print(minDistance.minDistanceSpaceOptimized("lemon", ""))
print(minDistance.minDistanceSpaceOptimized("prosperity", "properties"))
print(minDistance.minDistanceSpaceOptimized("spartan", "part"))

3
5
4
3


**312. Burst Balloons**


In [13]:
class Solution:
    # 27% time, 10% memory
    def maxCoins(self, nums: List[int]) -> int:
        """
        NeetCode video explanation:
        instead of choosing which index to pop next, choose which index to pop last
        This way the subproblems are subarrays not subsequences.
        n^2 subarrays vs 2^n subsequences
        """
        memo = {}  # {(left, right): coinAccumulated}

        def dfs(left, right) -> int:
            """
            Indices in the arguments are inclusive
            """
            if left > right:
                return 0
            elif (left, right) in memo:
                return memo[(left, right)]

            # break into subarrays on each side
            res = 0
            for iRemove in range(left, right + 1):
                l = dfs(left, iRemove - 1)
                r = dfs(iRemove + 1, right)
                # the third term is what you get when you finish popping the subarrays
                # if you pop nums[left: right+1] except nums[iRemove]
                # you will need to add nums[left-1] * nums[iRemove] * nums[right+1] to actually finish processing the case where iRemove is popped last
                res = max(
                    res, l + r + (nums[left - 1] * nums[iRemove] * nums[right + 1])
                )

            memo[(left, right)] = res
            return res

        nums = [1] + nums + [1]
        return dfs(1, len(nums) - 2)

    # 76% time, 61% memory
    def maxCoinsBottomUp(self, nums):
        n = len(nums)
        new_nums = [1] + nums + [1]

        dp = [[0] * (n + 2) for _ in range(n + 2)]
        # iterate through all subarrays whose left boundary is at `l` from 0 to n
        # (in reverse, so n to 0, because you need to compute the smallest windows first)
        for l in range(n, 0, -1):
            # iterate through all subarrays whose right boundary is at `r` (from `l` to n)
            for r in range(l, n + 1):
                # iterate through all values in that subarray
                for i in range(l, r + 1):
                    # the amount of coins you get for popping the ith index value LAST
                    coins = new_nums[l - 1] * new_nums[i] * new_nums[r + 1]
                    # the amount of coins you get for popping the left and right subarrays excluding the ith index value
                    coins += dp[l][i - 1] + dp[i + 1][r]
                    # update the max value of coins you can get for popping the subarray starting at `l` and ending at `r`
                    dp[l][r] = max(dp[l][r], coins)

        # 1 and n because of the padding added to the matrix
        return dp[1][n]


maxCoins = Solution()
print(maxCoins.maxCoinsBottomUp([3, 1, 5, 8]))
print(maxCoins.maxCoinsBottomUp([9, 76, 21]))
print(maxCoins.maxCoinsBottomUp([7, 9, 8, 0, 7, 1, 3, 5]))
print(maxCoins.maxCoinsBottomUp([2, 4, 8, 4, 0, 7, 8, 9, 1, 2, 4, 7, 1, 7, 3, 6]))

167
14574
1358
3304


**10. Regular Expression Matching**


In [14]:
"[[26,99,80,1,89,86,54,90,47,87],[9,59,61,49,14,55,77,3,83,79],[42,22,15,5,95,38,74,12,92,71],[58,40,64,62,24,85,30,6,96,52],[10,70,57,19,44,27,98,16,25,65],[13,0,76,32,29,45,28,69,53,41],[18,8,21,67,46,36,56,50,51,72],[39,78,48,63,68,91,34,4,11,31],[97,23,60,17,66,37,43,33,84,35],[75,88,82,20,7,73,2,94,93,81]]".replace(
    "[", "{"
).replace("]", "}")

'{{26,99,80,1,89,86,54,90,47,87},{9,59,61,49,14,55,77,3,83,79},{42,22,15,5,95,38,74,12,92,71},{58,40,64,62,24,85,30,6,96,52},{10,70,57,19,44,27,98,16,25,65},{13,0,76,32,29,45,28,69,53,41},{18,8,21,67,46,36,56,50,51,72},{39,78,48,63,68,91,34,4,11,31},{97,23,60,17,66,37,43,33,84,35},{75,88,82,20,7,73,2,94,93,81}}'

In [15]:
class Solution:
    # 76% time, 8% memory
    def isMatch(self, s: str, p: str) -> bool:
        """
        NeetCode cached recursion:

        Parameters
        - i index of p
        - j index of s

        If you encounter a character followed by an '*', the result of that branch depends on two options
        - At a position (i, j) where p[i + 1] is an "*", the answer is either of
            - (i+2, j) (no more repetition from asterisk)
            - (i, j+1) (one more repetition from asterisk)

        If the position is not followed by an '*', just check whether the letters match at p[i] and s[j]
        """
        n = len(p)
        m = len(s)
        cache = {}  # {(i,j) : bool}

        def dfs(i: int, j: int) -> bool:
            if (i, j) in cache:
                return cache[(i, j)]
            elif i >= n and j >= m:
                # we have exhausted both strings successfully
                return True
            elif i >= n:
                # if we have letters left unmatched in `s` we need to return False
                return False

            # check whether the current position matches since it is a reused comparison
            matches = j < m and (p[i] == "." or p[i] == s[j])

            # asterisk path:
            if i + 1 < n and p[i + 1] == "*":
                cache[(i, j)] = (
                    dfs(i + 2, j)  # no more asterisk repetition
                    or (
                        matches and dfs(i, j + 1)
                    )  # one more asterisk repetition if the letters match
                )
            elif matches:
                cache[(i, j)] = dfs(i + 1, j + 1)
            else:
                cache[(i, j)] = False

            return cache[(i, j)]

        return dfs(0, 0)

    # 96.6% time, 11% memory
    def isMatchBottomUp(self, s: str, p: str) -> bool:
        """
        Came up with this all on my own after implementing NeetCode's recursive solution.

        Try to turn the recursion into a recurrence relation

        Parameters
        - i index p
        - j index s

        Bounds
        - i [0, n)
        - j [0, m)

        Directions
        - both need to go in reverse.

        NOTE: Python does redundant operations when you use bitwise operators instead of boolean operators.
        - So, in my code I use `and` and `or`

        At `(i, j)`
        - if `p[i+1] == "*"`
            - `dp[i][j] = dp[i+2][j] | ( ((p[i] == s[j]) | (p[i] == ".")) & dp[i][j+1] )`
        - else
            - `dp[i][j] = ((p[i] == s[j]) | (p[i] == ".")) & dp[i+1][j+1]`

        How to handle out of bounds?
        - if they are both out of bounds then it will be true. So the bottom right corner can be extended by one square and set to True
        - if s has characters remaining but p doesn't it will be false so the bottom row can be extended by one square and set to False
        - if p has characters remaining but s doesn't it depends on the characters in p
            - I will handle these with if statements, not with an extra padding array
        """
        n = len(p)
        m = len(s)
        # padding the bottom with Falses because if you go out of bounds in i before j it is always False
        dp = [[False] * (m + 1) for _ in range(n + 1)]
        # if both out of bounds then True
        dp[n][m] = True

        for i in reversed(range(n)):
            if p[i] == "*":
                # no processing to be done if this is True
                continue

            for j in reversed(range(m + 1)):
                matches = j < m and (p[i] == "." or p[i] == s[j])

                if i + 1 < n and p[i + 1] == "*":
                    # if j < m you can check for the rightmost out of bounds column
                    # the rightmost out of bounds column is necessary because it replicates the case when
                    # the DFS in recursive solution kept repeating (i, j+1) -> (i, j+1) until j is out of bounds and it tries (i+2,j) which returns True when i+2 is out of bounds
                    # my rightmost column is essentially that, where the dp[n][m] offers a `True` when it is checked via dp[i+2][j] when j is out of bounds
                    # this then propagates the True via `matches and dp[i][j+1]`
                    dp[i][j] = dp[i + 2][j] or (matches and (j < m and dp[i][j + 1]))
                else:
                    dp[i][j] = matches and dp[i + 1][j + 1]

        # print("  ", "  ".join(s))
        # for i, row in enumerate(dp):
        #     print(p[i] if i < n else " ", row)
        return bool(dp[0][0])

    # 94.65% time, 9.88% memory
    def isMatchBottomUpSpaceOptimized(self, s: str, p: str) -> bool:
        """
        Trying to space optimize my earlier submission:
        I really only need two rows since I am skipping the rows where the index i corresponds to "*".
        This does mean that I need to replace the i+2 with i+1 assignments (I left most of my code comments from the earlier submission)
        """
        n = len(p)
        m = len(s)
        # starting with i == n (which is out of bounds) set to Falses because if you go out of bounds in i before j it is always False
        dp = [False] * (m + 1)
        # if both out of bounds == True
        dp[m] = True  # (i,j) == (n,m)

        for i in reversed(range(n)):
            dp = [False] * (m + 1)
            if p[i] == "*":
                # no processing to be done if this is True
                continue

            for j in reversed(range(m + 1)):
                matches = j < m and (p[i] == "." or p[i] == s[j])

                if i + 1 < n and p[i + 1] == "*":
                    # if j < m you can check for the rightmost out of bounds column
                    # the rightmost out of bounds column is necessary because it replicates the case when
                    # the DFS in recursive solution kept repeating (i, j+1) -> (i, j+1) until j is out of bounds and it tries (i+2,j) which returns True when i+2 is out of bounds
                    # my rightmost column is essentially that, where the dp[n][m] offers a `True` when it is checked via dp[i+2][j] when j is out of bounds
                    # this then propagates the True via `matches and dp[i][j+1]`
                    dp[j] = dp[j] or (matches and (j < m and dp[j + 1]))
                else:
                    dp[j] = matches and dp[j + 1]

            print(dp)
            dp = dp

        return dp[0]

    # 70% time, 17% memory
    def isMatchBottomUpSpaceOptimizedAgain(self, s: str, p: str) -> bool:
        """
        Trying to space optimize my earlier submission:
        I really only need two rows since I am skipping the rows where the index i corresponds to "*".
        This does mean that I need to replace the i+2 with i+1 assignments (I left most of my code comments from the earlier submission)

        Also for the ith index being processed you onle ever need dp[j] and dp[j+1] so i can get rid of the temporary array and just have two temporary variables
        """
        n = len(p)
        m = len(s)
        # starting with i == n (which is out of bounds) set to Falses because if you go out of bounds in i before j it is always False
        dp = [False] * (m + 1)
        # if both out of bounds == True
        dp[m] = True  # (i,j) == (n,m)

        for i in reversed(range(n)):
            if p[i] == "*":
                # no processing to be done if this is True
                continue

            # dp[i][j+1]
            dp_j1 = False

            for j in reversed(range(m + 1)):
                # dp[i][j]
                dp_j = dp[j]
                matches = j < m and (p[i] == "." or p[i] == s[j])

                if i + 1 < n and p[i + 1] == "*":
                    dp[j] = dp_j or (matches and (j < m and dp[j + 1]))
                else:
                    dp[j] = matches and dp_j1

                # update dp variables
                dp_j1 = dp_j

        return dp[0]


isMatch = Solution()
print(isMatch.isMatchBottomUpSpaceOptimizedAgain("aa", "a*"))
print(isMatch.isMatchBottomUpSpaceOptimizedAgain("aa", ".*.."))
print(isMatch.isMatchBottomUpSpaceOptimizedAgain("aab", "a*b"))
print(isMatch.isMatchBottomUpSpaceOptimizedAgain("mississippi", "mis*is*ip*."))
print(isMatch.isMatchBottomUpSpaceOptimizedAgain("abcd", "d*"))

True
True
True
True
False


**1639. Number of Ways to Form a Target String Given a Dictionary**


In [16]:
class Solution:
    # 98% time, 44% memory
    def numWays(self, words: List[str], target: str) -> int:
        """
        NOTE: the condition `if x in defaultdict` will always be True. This doubled my runtime in one of my earlier submissions.

        Observations
        ------------
        Since all words are of equal length, the separation of letters into words can be ignored.
        - The constraint that when an index `k` is used all previous indices for all words are unusable just means that for each index `k` in [0, length of any of the words] there are a list of letters available
        - First thing to do is turn `words` into `letters` where each index `k` maps to the letters available at that index.
        - Since the restriction is that all future letters must be at index `x > k`, we can just store a Counter of letters for each index
            - 0 < len(words) <= 1000, so a hashmap/Counter can be practical
            - the counter is useful because if there are 4 letter 'a' at an index `k` and from that point on there is n ways to reach targe. you multiply n by 4 when storing the result of (i,k)

        Parameters
        ----------
        - i index of `target`
        - k index of `letters`
        - x 'index + 1' of last-used letter in `letters` for the current path (DFS) we are on
            - the lower allowable bound of k in the DFS call

        Bounds
        ------
        - i [0, len(target))
        - k [x, len(letters))

        I think this question is a lot easier top down with memo lol which I will do first
        """
        MOD = 10**9 + 7
        K = len(words[0])
        n = len(target)
        a = ord("a")
        letters = [[0] * 26 for _ in range(K)]
        for word in words:
            for k in range(K):
                letters[k][ord(word[k]) - a] += 1

        cache = {}  # { (i,x): num ways from position (i,x) } x being the lower limit of k (inclusive)

        def dfs(i: int, x: int) -> int:
            if (i, x) in cache:
                return cache[(i, x)]
            # think of the base cases:
            elif i == n:
                # if you complete target, then you return 1 because a singular and unique DFS path completed
                return 1
            elif x > K - (n - i):
                # if you run out of letters early exit
                return 0

            # now the actual DFS:
            if not letters[x][ord(target[i]) - a]:
                # if the characters don't match: increment x but don't increment i
                res = dfs(i, x + 1)
            else:
                # if letters match, same as above but also add the path where you increment x and i
                # i purposefully order it like this so that the i-increment path can fill some cache space
                res = (letters[x][ord(target[i]) - a] * dfs(i + 1, x + 1)) + (
                    dfs(i, x + 1)
                )

            cache[(i, x)] = res % MOD
            return cache[(i, x)]

        # question asks to return the answer modulo 10^9 + 7
        return dfs(0, 0)

    # 92% time, 51% memory
    def numWaysBottomUp(self, words: List[str], target: str) -> int:
        """
        Trying to implement bottom up version of my memo solution
        """
        MOD = 10**9 + 7
        K = len(words[0])
        n = len(target)
        a = ord("a")
        letters = [[0] * 26 for _ in range(K)]
        for word in words:
            for k in range(K):
                letters[k][ord(word[k]) - a] += 1

        # padding the bottom row (target out of bounds) with 1 since if you reach it it means your path is successful
        # padding the rightmost column as well, because you can succeed with both variables out of bounds
        #   go back and read the DFS base cases for clarity
        # the only one idk how to implement off rip is `if x > K - (n - i): return 0`
        #   I guess I can just modify the iteration bounds, which means x has to be the inner loop
        dp = [[0] * (K + 1) for _ in range(n)]
        dp.append([1] * (K + 1))

        for i in reversed(range(n)):
            idx = ord(target[i]) - a
            # see my comment blurb right above for an explanation on the bounds of iteration
            for x in reversed(range(K - (n - i) + 1)):
                if not letters[x][idx]:
                    # if the characters don't match: increment x but don't increment i
                    res = dp[i][x + 1]
                else:
                    # if letters match, same as above but also add the path where you increment x and i
                    # i purposefully order it like this so that the i-increment path can fill some cache space
                    res = (letters[x][idx] * dp[i + 1][x + 1]) + (dp[i][x + 1])

                # question asks to return the answer modulo 10^9 + 7
                dp[i][x] = res % MOD

        return dp[0][0]

    # 96% time, 76% memory
    def numWaysBottomUpSpaceOptimized(self, words: List[str], target: str) -> int:
        """
        Trying to implement bottom up version of my memo solution
        Space Op: 2 arrays instead of matrix
        """
        MOD = 10**9 + 7
        K = len(words[0])
        n = len(target)
        a = ord("a")
        letters = [[0] * 26 for _ in range(K)]
        for word in words:
            for k in range(K):
                letters[k][ord(word[k]) - a] += 1

        dpIP1 = [1] * (K + 1)

        for i in reversed(range(n)):
            dpI = [0] * (K + 1)
            idx = ord(target[i]) - a
            for x in reversed(range(K - (n - i) + 1)):
                if not letters[x][idx]:
                    dpI[x] = dpI[x + 1] % MOD
                else:
                    dpI[x] = ((letters[x][idx] * dpIP1[x + 1]) + (dpI[x + 1])) % MOD

            dpIP1 = dpI
        return dpI[0]


numWays = Solution()
print(numWays.numWaysBottomUpSpaceOptimized(["acca", "bbbb", "caca"], "aba"))
print(numWays.numWaysBottomUpSpaceOptimized(["acca", "bbbb", "caca"], "aba"))
print(numWays.numWaysBottomUpSpaceOptimized(["abba", "baab"], "bab"))
print(numWays.numWaysBottomUpSpaceOptimized(["b"], "a"))

6
6
4
0


**689. Maximum Sum of 3 Non-Overlapping Subarrays**


In [34]:
class Solution:
    # 97% time, 63% memory
    def maxSumOfThreeSubarrays(self, nums: List[int], k: int) -> List[int]:
        """
        Can you divide and conquer?
        for a set of 3 pointers you compute their k window sum and store the result in a global cache
        then move the rightmost pointer to the right, then the middle, then

        Why can't you just compute the sum of every k-length subarray
        then do the 3 pointers

        Nah this takes too long, I can definitely use a rightMaxSuffix if I iterate in reverse
        If I start i,j,h at (n-3k), (n-2k), (n-k)
        - then I store the max seen for each level

        Whenver I iterate all three pointers backwards
        - for (h)
            - I can set the new max_h as `max(k_sums[h], max_h)`
        - for (j,h)
            - I can set the new max_j_h as `max(k_sums[j] + max_h, max_j_h)`
        - for (i,j,h)
            - I can set the new max_i_j_h as `max(k_sums[i] + max_j_h, max_i_j_h)`
        - same logic for any larger amount of pointers
        """
        n = len(nums)
        n_effective = n - k + 1

        k_sums = [0] * (n_effective)
        k_sums[0] = sum(nums[:k])
        for left in range(1, n_effective):
            k_sums[left] = k_sums[left - 1] - nums[left - 1] + nums[left + k - 1]

        i, j, h = n_effective - 2 * k - 1, n_effective - k - 1, n_effective - 1

        max_h: Tuple[int, Tuple[int]] = (k_sums[h], (h,))
        max_j_h: Tuple[int, Tuple[int, int]] = (k_sums[j] + max_h[0], (j, h))
        max_i_j_h: Tuple[int, Tuple[int, int, int]] = (
            k_sums[i] + max_j_h[0],
            (i, j, h),
        )

        # the max number of iterations I can do while shifting the 3 pointers to the left
        for offset in range(n_effective - 2 * k):
            # i check for equality because we are iterating the pointer in reverse and we want lexicograph sort
            # so the later we see a max the better the higher priority the lexicographical sort

            # resolve h
            if k_sums[h - offset] >= max_h[0]:
                max_h = (k_sums[h - offset], (h - offset,))
            # resolve j
            if k_sums[j - offset] + max_h[0] >= max_j_h[0]:
                max_j_h = (k_sums[j - offset] + max_h[0], (j - offset, *max_h[1]))
            # resolve i
            if k_sums[i - offset] + max_j_h[0] >= max_i_j_h[0]:
                max_i_j_h = (k_sums[i - offset] + max_j_h[0], (i - offset, *max_j_h[1]))

        return list(max_i_j_h[1])


maxSumOfThreeSubarrays = Solution()
print(maxSumOfThreeSubarrays.maxSumOfThreeSubarrays([1, 2, 1, 2, 6, 7, 5, 1], 2))

[0, 3, 5]


In [ ]:
class Solution:
    def shortestCommonSupersequence(self, str1: str, str2: str) -> str:
        """
        This is my attempt (done after copying neetcode's solutions below)
        There were 2 solutions possible:
        - Find the Longest Common Subsequence and then Add the remaining letters in the correct order
        - Actually do the Bottom Up DP where you store the shortes supersequence for each index pair (i,j)

        BU DP

        Parameters:
        - i index of str1
        - j index of str2

        Choices:
        - Add str1[i] to existing supersequence
        - Add str2[j] to existing supersequence
        - Can be done at the same time if they are the same value

        Directions:
        i and j from big to small means we look diagonally down and to the right dp[i+1][j+1] for recurrence
        i and j from small to big means we look diagonally up and to the left dp[i-1][j-1] for recurrence

        Check the last method of this Solution class (it space optimized 1 dimension)

        LCS + Extra:


        """
        return ""

    def shortestCommonSupersequenceBacktrack(self, str1: str, str2: str) -> str:
        """
        Caching the backtracking topdown fails on the last test case for MLE
        O(n * m * (n + m)) time and memory
        """
        N, M = len(str1), len(str2)
        # 2d grid to store the string results (won't be sparse so grid works)
        cache = [["" for _ in range(M)] for _ in range(N)]

        def backtrack(i, j) -> str:
            if i == len(str1):  # success base cases
                return str2[j:]
            elif j == len(str2):  # success base cases
                return str1[i:]
            elif cache[i][j]:
                return cache[i][j]

            if str1[i] == str2[j]:  # can skip a character for both
                return str1[i] + backtrack(i + 1, j + 1)

            # take from str1
            res1 = str1[i] + backtrack(i + 1, j)
            # take from str2
            res2 = str2[j] + backtrack(i, j + 1)
            if len(res1) < len(res2):  # return shorter string
                cache[i][j] = res1
                return res1
            cache[i][j] = res2
            return res2

        return backtrack(0, 0)

    # 24% time, 95% memory
    def shortestCommonSupersequenceBottomUp(self, str1: str, str2: str) -> str:
        """bottom up of the backtracking solution above"""
        N, M = len(str1), len(str2)
        # init the previous row as dp[N] (none of str1 is included)
        prev = [str2[j:] for j in range(M)]
        prev.append("")
        for i in reversed(range(N)):
            cur = [""] * M
            cur.append(str1[i:])
            for j in reversed(range(M)):
                if str1[i] == str2[j]:
                    cur[j] = str1[i] + prev[j + 1]  # diagonal below to the right
                else:
                    res1 = str1[i] + prev[j]
                    res2 = str2[j] + cur[j + 1]
                    if len(res1) < len(res2):
                        cur[j] = res1
                    else:
                        cur[j] = res2
            prev = cur
        return cur[0]

**2999. Count the Number of Powerful Integers**

I couldn't solve this, and I don't understand either of the solutions.
Will have to come back to this in the future.


In [ ]:
class SolutionDP:
    def numberOfPowerfulInt(self, start: int, finish: int, limit: int, s: str) -> int:
        low = str(start)
        high = str(finish)
        n = len(high)
        low = low.zfill(n)  # align digits
        pre_len = n - len(s)  # prefix length

        @cache
        def dfs(i, limit_low, limit_high):
            # recursive boundary
            if i == n:
                return 1
            lo = int(low[i]) if limit_low else 0
            hi = int(high[i]) if limit_high else 9
            res = 0
            if i < pre_len:
                for digit in range(lo, min(hi, limit) + 1):
                    res += dfs(
                        i + 1,
                        limit_low and digit == lo,
                        limit_high and digit == hi,
                    )
            else:
                x = int(s[i - pre_len])
                if lo <= x <= min(hi, limit):
                    res = dfs(i + 1, limit_low and x == lo, limit_high and x == hi)

            return res

        return dfs(0, True, True)


class SolutionCombinatorics:
    def numberOfPowerfulInt(self, start: int, finish: int, limit: int, s: str) -> int:
        start_ = str(start - 1)
        finish_ = str(finish)
        return self.calculate(finish_, s, limit) - self.calculate(start_, s, limit)

    def calculate(self, x: str, s: str, limit: int) -> int:
        if len(x) < len(s):
            return 0
        if len(x) == len(s):
            return 1 if x >= s else 0

        suffix = x[len(x) - len(s) :]
        count = 0
        pre_len = len(x) - len(s)

        for i in range(pre_len):
            if limit < int(x[i]):
                count += (limit + 1) ** (pre_len - i)
                return count
            count += int(x[i]) * (limit + 1) ** (pre_len - 1 - i)

        if suffix >= s:
            count += 1

        return count